# Principal Component Analysis (PCA) Tutorial

This notebook covers the following topics:
1. Introduction to Dimensionality Reduction and PCA
2. Using Scikit-Learn's PCA implementation
3. Understanding PCA Variance Ratio
4. Randomized PCA
5. Incremental PCA
6. PCA on the MNIST dataset

Let's get started!

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA, IncrementalPCA, RandomizedPCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml, make_blobs
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# Set random seed for reproducibility
np.random.seed(42)

## 1. Introduction to Dimensionality Reduction and PCA

### What is Dimensionality Reduction?

Dimensionality reduction refers to techniques that reduce the number of features in a dataset while preserving as much information as possible. These techniques are useful for:

- Reducing computational complexity
- Visualizing high-dimensional data
- Removing redundant or correlated features
- Reducing overfitting by simplifying the model
- Addressing the "curse of dimensionality"

### What is PCA?

Principal Component Analysis (PCA) is one of the most popular dimensionality reduction techniques. It works by:

1. Finding directions (principal components) in the data that maximize variance
2. Projecting the data onto these principal components
3. Using only the top N components to represent the data in lower dimensions

The principal components are orthogonal to each other, and they are ordered by the amount of variance they explain in the data.

Let's demonstrate PCA with a simple 2D dataset first, so we can visualize what's happening:

In [ ]:
# Generate a simple 2D dataset with correlation
np.random.seed(42)
n_samples = 500

# Create correlated data
X = np.random.randn(n_samples, 2)
transformation = [[0.8, -0.6], [0.6, 0.8]]
X = np.dot(X, transformation)

# Plot the data
plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 1], alpha=0.7)
plt.title('Original 2D Dataset with Correlation')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True)
plt.axis('equal')
plt.show()

In [ ]:
# Apply PCA to find principal components
pca = PCA()
pca.fit(X)

# Get the principal components
components = pca.components_
explained_variance = pca.explained_variance_

# Plot the data and principal components
plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 1], alpha=0.7)

# Plot the principal components
for i, (component, variance) in enumerate(zip(components, explained_variance)):
    plt.arrow(0, 0, component[0] * np.sqrt(variance), component[1] * np.sqrt(variance),
              head_width=0.1, head_length=0.1, fc=f'C{i+2}', ec=f'C{i+2}',
              label=f'PC{i+1} (var={variance:.2f})')

plt.title('Dataset with Principal Components')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True)
plt.axis('equal')
plt.legend()
plt.show()

# Print the explained variance ratio
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {np.sum(pca.explained_variance_ratio_):.4f}")

## 2. Using Scikit-Learn's PCA Implementation

Scikit-Learn makes it easy to apply PCA to your data. Let's walk through the standard workflow:

In [ ]:
# Create a more complex dataset
X, _ = make_blobs(n_samples=1000, n_features=10, centers=5, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=3)  # Reduce to 3 dimensions
X_pca = pca.fit_transform(X_scaled)

print(f"Original shape: {X.shape}")
print(f"Reduced shape: {X_pca.shape}")

In [ ]:
# Visualize the transformed data (first 2 principal components)
plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.7)
plt.title('Data projected onto first two principal components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()

### Key Parameters of Scikit-Learn's PCA

- `n_components`: Number of components to keep
  - Integer value: Keep specific number of components
  - Float value between 0 and 1: Keep enough components to explain that ratio of variance
  - 'mle': Use automatic dimensionality selection
- `svd_solver`: Method for SVD calculation
  - 'auto': Auto-select based on data size
  - 'full': Run exact full SVD
  - 'arpack': Run randomized SVD
  - 'randomized': Use randomized algorithm
- `whiten`: Boolean, whether to whiten the data (normalize components to have unit variance)
- `random_state`: Controls the randomization in the algorithm (if applicable)

## 3. Understanding PCA Variance Ratio

The explained variance ratio tells us how much of the total variance in the dataset is explained by each principal component. It's a key metric for determining how many components to keep.

In [ ]:
# Apply PCA without limiting components
pca = PCA()
pca.fit(X_scaled)

# Plot the explained variance ratio
plt.figure(figsize=(12, 6))

# Individual explained variance
plt.subplot(1, 2, 1)
plt.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance by Component')
plt.xticks(range(1, len(pca.explained_variance_ratio_) + 1))

# Cumulative explained variance
plt.subplot(1, 2, 2)
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance')
plt.xticks(range(1, len(pca.explained_variance_ratio_) + 1))
plt.axhline(y=0.9, color='r', linestyle='--', label='90% Variance')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Find number of components to explain 95% of variance
cumsum = np.cumsum(pca.explained_variance_ratio_)
n_components_95 = np.argmax(cumsum >= 0.95) + 1
print(f"Number of components needed to explain 95% of variance: {n_components_95}")

# Apply PCA with variance ratio
pca_95 = PCA(n_components=0.95)  # Keep components that explain 95% of variance
X_pca_95 = pca_95.fit_transform(X_scaled)
print(f"Original shape: {X.shape}")
print(f"Reduced shape: {X_pca_95.shape}")

### Using Explained Variance Ratio to Select Number of Components

There are several strategies to select the number of components:

1. **Explained Variance Threshold**: Choose components that explain a certain percentage of variance (e.g., 95%).
2. **Elbow Method**: Look for the "elbow" in the plot, where adding more components results in diminishing returns.
3. **Kaiser Rule**: Keep components with eigenvalues (or explained variance) greater than 1.

## 4. Randomized PCA

For large datasets, computing the full PCA can be computationally expensive. Randomized PCA uses approximation techniques to speed up the calculation.

In [ ]:
# In newer scikit-learn versions, RandomizedPCA is deprecated
# Instead, use PCA with svd_solver='randomized'

# Generate a larger dataset
X_large, _ = make_blobs(n_samples=10000, n_features=100, centers=10, random_state=42)
X_large_scaled = StandardScaler().fit_transform(X_large)

# Time the standard PCA
import time

start_time = time.time()
pca_full = PCA(n_components=10, svd_solver='full')
X_pca_full = pca_full.fit_transform(X_large_scaled)
full_time = time.time() - start_time
print(f"Full PCA time: {full_time:.4f} seconds")

# Time the randomized PCA
start_time = time.time()
pca_random = PCA(n_components=10, svd_solver='randomized', random_state=42)
X_pca_random = pca_random.fit_transform(X_large_scaled)
random_time = time.time() - start_time
print(f"Randomized PCA time: {random_time:.4f} seconds")
print(f"Speedup: {full_time/random_time:.2f}x")

In [ ]:
# Compare variance explained
plt.figure(figsize=(10, 6))
plt.bar(range(1, 11), pca_full.explained_variance_ratio_, alpha=0.5, label='Full PCA')
plt.bar(range(1, 11), pca_random.explained_variance_ratio_, alpha=0.5, label='Randomized PCA')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Full PCA vs Randomized PCA: Explained Variance')
plt.legend()
plt.xticks(range(1, 11))
plt.show()

### When to Use Randomized PCA

- For large datasets (many samples or features)
- When computation time is a concern
- When you need only a few principal components
- When an approximate solution is acceptable

Key parameters for randomized PCA:
- `n_components`: Number of components
- `random_state`: Controls randomization for reproducibility
- `iterated_power`: Number of power iterations (higher values improve accuracy but take longer)

## 5. Incremental PCA

Incremental PCA (IPCA) is designed for datasets that are too large to fit in memory. It processes the data in batches, making it suitable for online learning or large-scale applications.

In [ ]:
# Simulate a large dataset using batches
X_huge, _ = make_blobs(n_samples=20000, n_features=50, centers=15, random_state=42)
X_huge_scaled = StandardScaler().fit_transform(X_huge)

# Apply Incremental PCA
n_batches = 10
batch_size = X_huge.shape[0] // n_batches

ipca = IncrementalPCA(n_components=10)

# Process data in batches
start_time = time.time()
for i in range(n_batches):
    batch = X_huge_scaled[i * batch_size:(i + 1) * batch_size]
    ipca.partial_fit(batch)

ipca_time = time.time() - start_time
print(f"Incremental PCA time: {ipca_time:.4f} seconds")

# Transform the data
X_ipca = ipca.transform(X_huge_scaled)

In [ ]:
# Compare with regular PCA
start_time = time.time()
pca_regular = PCA(n_components=10)
X_pca_regular = pca_regular.fit_transform(X_huge_scaled)
regular_time = time.time() - start_time
print(f"Regular PCA time: {regular_time:.4f} seconds")

# Compare explained variance
plt.figure(figsize=(10, 6))
plt.bar(range(1, 11), pca_regular.explained_variance_ratio_, alpha=0.5, label='Regular PCA')
plt.bar(range(1, 11), ipca.explained_variance_ratio_, alpha=0.5, label='Incremental PCA')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Regular PCA vs Incremental PCA: Explained Variance')
plt.legend()
plt.xticks(range(1, 11))
plt.show()

### When to Use Incremental PCA

- For datasets too large to fit in memory
- When data arrives in a streaming manner
- In online learning scenarios
- When working with very large datasets on machines with limited memory

Key parameters for Incremental PCA:
- `n_components`: Number of components
- `batch_size`: Size of batches for the incremental fit
- `whiten`: Whether to whiten the output

The `partial_fit()` method allows for processing data in batches.

## 6. PCA on the MNIST Dataset

Now, let's apply PCA to a real-world dataset: MNIST handwritten digits. This demonstrates how PCA can be used for dimensionality reduction in image data.

In [ ]:
# Download MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
X_mnist = mnist.data.astype('float32')
y_mnist = mnist.target.astype('int')

# Scale the data
X_mnist_scaled = X_mnist / 255.0

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_mnist_scaled, y_mnist, test_size=0.2, random_state=42)

print(f"MNIST data shape: {X_mnist.shape}")
print(f"Each image is a vector of length 784 (28x28 pixels)")

In [ ]:
# Visualize some MNIST digits
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.ravel()

for i in range(10):
    idx = np.where(y_train == str(i))[0][0]
    axes[i].imshow(X_train[idx].reshape(28, 28), cmap='gray')
    axes[i].set_title(f"Digit: {y_train[idx]}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Apply PCA to MNIST
n_components = 100  # We'll keep 100 components initially
pca_mnist = PCA(n_components=n_components)
X_train_pca = pca_mnist.fit_transform(X_train)
X_test_pca = pca_mnist.transform(X_test)

print(f"Original dimensions: {X_train.shape[1]}")
print(f"Reduced dimensions: {X_train_pca.shape[1]}")
print(f"Dimensionality reduction: {X_train.shape[1]/X_train_pca.shape[1]:.1f}x")

In [ ]:
# Plot explained variance ratio
plt.figure(figsize=(12, 6))

# Cumulative explained variance
plt.plot(np.cumsum(pca_mnist.explained_variance_ratio_), marker='o', markersize=3)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('MNIST: Cumulative Explained Variance')

# Add lines at 80%, 90%, 95% variance
for threshold, color, label in zip([0.8, 0.9, 0.95], ['r', 'g', 'b'], ['80%', '90%', '95%']):
    n_components = np.argmax(np.cumsum(pca_mnist.explained_variance_ratio_) >= threshold) + 1
    plt.axhline(y=threshold, color=color, linestyle='--', alpha=0.5, label=f'{label} variance ({n_components} components)')
    plt.axvline(x=n_components, color=color, linestyle='--', alpha=0.5)

plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the first 25 principal components
fig, axes = plt.subplots(5, 5, figsize=(12, 12))
axes = axes.ravel()

for i in range(25):
    component = pca_mnist.components_[i].reshape(28, 28)
    axes[i].imshow(component, cmap='viridis')
    axes[i].set_title(f"PC {i+1}")
    axes[i].axis('off')

plt.suptitle("First 25 Principal Components of MNIST", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

### Image Reconstruction with PCA

One interesting application of PCA is reconstructing the original data from the reduced dimensions. Let's see how the reconstructed MNIST digits look with different numbers of components:

In [ ]:
# Function to reconstruct from PCA
def reconstruct_from_pca(X, pca, n_components=None):
    pca_obj = PCA(n_components=n_components)
    pca_obj.fit(X)
    X_reduced = pca_obj.transform(X)
    X_reconstructed = pca_obj.inverse_transform(X_reduced)
    return X_reconstructed, pca_obj.explained_variance_ratio_.sum()

# Select a test example
digit_idx = np.where(y_test == '5')[0][0]  # Find a '5' digit
original_digit = X_test[digit_idx]

# Reconstruct with different numbers of components
n_components_list = [5, 10, 20, 50, 100, 200]
reconstructed_digits = []
explained_variances = []

for n in n_components_list:
    X_reconstructed, variance = reconstruct_from_pca(X_test, pca_mnist, n)
    reconstructed_digits.append(X_reconstructed[digit_idx])
    explained_variances.append(variance)

In [ ]:
# Visualize original and reconstructed digits
fig, axes = plt.subplots(1, len(n_components_list) + 1, figsize=(16, 3))

# Plot original
axes[0].imshow(original_digit.reshape(28, 28), cmap='gray')
axes[0].set_title("Original")
axes[0].axis('off')

# Plot reconstructions
for i, (n, rec_digit, variance) in enumerate(zip(n_components_list, reconstructed_digits, explained_variances)):
    axes[i+1].imshow(rec_digit.reshape(28, 28), cmap='gray')
    axes[i+1].set_title(f"{n} Components\n{variance:.2%} Variance")
    axes[i+1].axis('off')

plt.suptitle("MNIST Digit Reconstruction with PCA", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

### Using PCA for Classification of MNIST

Let's see how PCA can be used as a preprocessing step for classification.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Create pipelines with different numbers of PCA components
component_options = [20, 50, 100, 200]
results = []

for n_components in component_options:
    # Create pipeline with PCA and SVM
    pipeline = Pipeline([
        ('pca', PCA(n_components=n_components)),
        ('svm', SVC(kernel='rbf', gamma='scale'))
    ])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Evaluate
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results.append((n_components, accuracy))
    print(f"PCA with {n_components} components: {accuracy:.4f} accuracy")

In [ ]:
# Plot the results
components, accuracies = zip(*results)

plt.figure(figsize=(10, 6))
plt.plot(components, accuracies, marker='o', linestyle='-')
plt.title('Classification Accuracy vs. Number of PCA Components')
plt.xlabel('Number of PCA Components')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

## 7. PCA in a Machine Learning Pipeline

Scikit-Learn's Pipeline makes it easy to include PCA as a preprocessing step in your machine learning workflow.

In [ ]:
# Create a pipeline with standardization, PCA, and a classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=100)),
    ('classifier', SVC(kernel='rbf', gamma='scale'))
])

# Parameters to search
param_grid = {
    'pca__n_components': [50, 100, 150],
    'classifier__C': [0.1, 1, 10]
}

# Use GridSearchCV to find the best parameters
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Print the best parameters
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV score: {grid_search.best_score_:.4f}")

# Evaluate on the test set
y_pred = grid_search.predict(X_test)
print(f"Test set accuracy: {accuracy_score(y_test, y_pred):.4f}")

## 8. Practical Tips for Using PCA

### When to Use PCA

- **High-dimensional data**: When you have many features (especially if many are correlated)
- **Visualization**: To reduce dimensionality for visualization (2D or 3D plots)
- **Noise reduction**: To filter out noise by discarding less important components
- **Speed/efficiency**: To speed up machine learning algorithms
- **Multicollinearity**: To address multicollinearity in regression problems

### When Not to Use PCA

- **Interpretability is critical**: PCA components are not easily interpretable
- **Feature importance is needed**: PCA transforms features, making it hard to identify importance of original features
- **Sparse data**: Other methods like sparse PCA might be more appropriate
- **Non-linear relationships**: PCA captures linear relationships; consider kernel PCA or t-SNE for non-linear data

### Best Practices

1. **Standardize data**: Always standardize/normalize data before applying PCA
2. **Check variance explained**: Look at cumulative variance to determine number of components
3. **Beware of outliers**: PCA is sensitive to outliers
4. **Cross-validate**: When using PCA in a pipeline, cross-validate to find optimal parameters
5. **Consider the problem**: For some problems, domain-specific feature engineering might be better than PCA

## 9. Comparison with Other Dimensionality Reduction Techniques

PCA is just one of many dimensionality reduction techniques. Here's a brief comparison with other methods:

In [ ]:
# Compare PCA with t-SNE on MNIST
from sklearn.manifold import TSNE

# Use a subset for speed
n_samples = 2000
X_subset = X_train[:n_samples]
y_subset = y_train[:n_samples]

# Apply PCA
pca_vis = PCA(n_components=2)
X_pca_vis = pca_vis.fit_transform(X_subset)

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_subset)

# Create comparison plot
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Plot PCA
for digit in range(10):
    idx = y_subset == str(digit)
    axes[0].scatter(X_pca_vis[idx, 0], X_pca_vis[idx, 1], alpha=0.7, label=f"Digit {digit}")
axes[0].set_title('PCA: MNIST Digits')
axes[0].set_xlabel('Principal Component 1')
axes[0].set_ylabel('Principal Component 2')
axes[0].legend()
axes[0].grid(True)

# Plot t-SNE
for digit in range(10):
    idx = y_subset == str(digit)
    axes[1].scatter(X_tsne[idx, 0], X_tsne[idx, 1], alpha=0.7, label=f"Digit {digit}")
axes[1].set_title('t-SNE: MNIST Digits')
axes[1].set_xlabel('t-SNE Component 1')
axes[1].set_ylabel('t-SNE Component 2')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

### Comparison of Dimensionality Reduction Techniques

| Technique | Strengths | Weaknesses | Use Cases |
|-----------|-----------|------------|------------|
| **PCA** | Fast, simple, well-understood<br>Preserves global structure<br>Handles high-dimensional data | Linear relationships only<br>Sensitive to scaling<br>Components not easily interpretable | General dimensionality reduction<br>Preprocessing step<br>Noise reduction |
| **t-SNE** | Preserves local structure<br>Better separation of clusters<br>Works well for visualization | Slow on large datasets<br>Stochastic results<br>Not suitable for preprocessing | Visualization<br>Exploring cluster structure<br>Non-linear data |
| **UMAP** | Faster than t-SNE<br>Preserves global & local structure<br>Theoretical foundation | Complex parameter tuning<br>Newer, less well-understood | Visualization<br>Alternative to t-SNE<br>Large datasets |
| **Kernel PCA** | Captures non-linear relationships<br>Extends PCA framework<br>Theoretical foundation | Computationally expensive<br>Difficult kernel selection<br>Hard to interpret | Non-linear data<br>When linear PCA fails |
| **Factor Analysis** | Statistical model<br>Focuses on correlations<br>Good for latent factors | Assumes linear relationships<br>Needs distributional assumptions | Social sciences<br>Psychology<br>Market research |
| **Autoencoders** | Very flexible<br>Can capture complex patterns<br>Non-linear capabilities | Requires neural network expertise<br>Computationally expensive<br>Risk of overfitting | Complex data<br>Image processing<br>Deep learning |

## 10. Conclusion

In this notebook, we've covered:

1. The fundamentals of PCA and dimensionality reduction
2. How to use Scikit-Learn's PCA implementation
3. The importance of explained variance ratio in selecting components
4. Randomized PCA for large datasets
5. Incremental PCA for out-of-memory data processing
6. Application of PCA to the MNIST dataset
7. Including PCA in a machine learning pipeline
8. Practical tips for using PCA effectively
9. Comparison with other dimensionality reduction techniques

Principal Component Analysis is a powerful technique for dimensionality reduction with applications across many domains. It provides a balance of simplicity, efficiency, and effectiveness that makes it a standard tool in any data scientist's toolkit.

Remember, while PCA is excellent for many applications, it's always important to consider the specific needs of your problem and data to determine if it's the right tool for the job.